# Custom Tracing with the W&B Trace Decorator

<a target="_blank" href="https://colab.research.google.com/github/wandb/weave/blob/master/examples/prompts/trace_debugging/trace_quickstart_decorator.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Step 0: Setup

Install dependencies and login to W&B so you can save and share your work.

In [ ]:
!pip install -qqq weave wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
import weave
import time
from weave.legacy.monitoring import init_monitor

# Step 1: Configure data streaming and storage in W&B

In [ ]:
WB_PROJECT = "traces"
WB_STREAM = "decorator_traces_stream"
WB_ENTITY = "" # optional: wandb username or team name

**Note**: The WB_ENTITY will match the default entity associated with your [W&B API key](wandb.ai/authorize). You can optionally set WB_ENTITY to a different wandb username or team name. Log in to W&B and navigate to [the Home Page](https://wandb.ai/home) to see any other valid options for your WB_ENTITY under your "Profile" and "Teams" in the left sidebar.

# Step 2: Setup log stream and add decorators to functions of interest

In [ ]:
mon = init_monitor(f"{WB_ENTITY}/{WB_PROJECT}/{WB_STREAM}")

# Wrap a function to make it auto-log
@mon.trace()
def adder(a, b):
    time.sleep(0.2)
    return a + b

@mon.trace()
def buggy_fn(a, b):
    time.sleep(0.2)
    raise Exception("Intentional example exception happening here :)")
    return a * b

# you can create manual spans with the mon.span context manager
with mon.span('a_span') as s:
    time.sleep(0.5)
    # you can add attributes to all child spans with the mon.attributes context manager
    with mon.attributes({'a': 5, 'b': 6}):
        # spans can be nested
        with mon.span('b_span') as b:
            time.sleep(0.5)
            # call the traced function to generate a span for it
            # we add an argument "monitor_attributes" which allows you pass in additional
            # attributes to be logged
            result = adder(1, 2, monitor_attributes={'j': 15})
            buggy_fn(result, 2)
        time.sleep(0.05)

# Step 3: Create and customize a Trace Debugging Board

Click on the "View data at:" link above to view your data stream in Weave. 
You can click on "+ New board from template" on the right to create a Trace Debug Board to track, dynamically analyze, and understand all your decorated calls.